***GENERATED CODE FOR washingtonians PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateAware', 'transformation_label': 'String Indexer'}], 'feature': 'DateAware', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '479', 'mean': '', 'stddev': '', 'min': '1/1/2020', 'max': '9/9/2020', 'missing': '21', 'distinct': '319'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DateAware'}, {'feature_label': 'DateAware', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DateAware')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateSubmitted', 'transformation_label': 'String Indexer'}], 'feature': 'DateSubmitted', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/10/2018', 'max': '9/9/2020', 'missing': '0', 'distinct': '340'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DateSubmitted'}, {'feature_label': 'DateSubmitted', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DateSubmitted')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataBreachCause', 'transformation_label': 'String Indexer'}], 'feature': 'DataBreachCause', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Cyberattack', 'max': 'Unauthorized Access', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataBreachCause'}, {'feature_label': 'DataBreachCause', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataBreachCause')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateStart', 'transformation_label': 'String Indexer'}], 'feature': 'DateStart', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '417', 'mean': '', 'stddev': '', 'min': '1/1/2020', 'max': '9/7/2019', 'missing': '83', 'distinct': '266'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DateStart'}, {'feature_label': 'DateStart', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DateStart')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateEnd', 'transformation_label': 'String Indexer'}], 'feature': 'DateEnd', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '383', 'mean': '', 'stddev': '', 'min': '1/1/2017', 'max': '9/7/2020', 'missing': '117', 'distinct': '223'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DateEnd'}, {'feature_label': 'DateEnd', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DateEnd')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Name', 'transformation_label': 'String Indexer'}], 'feature': 'Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '20/20 Eye Care Network', 'max': 'thyssenkrupp Materials', 'missing': '0', 'distinct': '492'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Name'}, {'feature_label': 'Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Id', 'transformation_label': 'String Indexer'}], 'feature': 'Id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '10119.42', 'stddev': '341.04', 'min': ' FSB', 'max': '9775', 'missing': '0', 'distinct': '368'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Id'}, {'feature_label': 'Id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CyberattackType', 'transformation_label': 'String Indexer'}], 'feature': 'CyberattackType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '424', 'mean': '10123.67', 'stddev': '359.61', 'min': ' Cherry River Farms', 'max': 'Unclear/unknown', 'missing': '76', 'distinct': '167'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CyberattackType'}, {'feature_label': 'CyberattackType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CyberattackType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WashingtoniansAffected', 'transformation_label': 'String Indexer'}], 'feature': 'WashingtoniansAffected', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '440', 'mean': '28750.05', 'stddev': '138559.37', 'min': '1000', 'max': 'Unclear/unknown', 'missing': '60', 'distinct': '327'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'WashingtoniansAffected'}, {'feature_label': 'WashingtoniansAffected', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('WashingtoniansAffected')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'IndustryType', 'transformation_label': 'String Indexer'}], 'feature': 'IndustryType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '490', 'mean': '29644.42', 'stddev': '267067.77', 'min': '1012', 'max': 'Ransomware', 'missing': '10', 'distinct': '152'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'IndustryType'}, {'feature_label': 'IndustryType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('IndustryType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BusinessType', 'transformation_label': 'String Indexer'}], 'feature': 'BusinessType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '283', 'mean': '1369.5', 'stddev': '824.39', 'min': '1690', 'max': 'Web Services', 'missing': '217', 'distinct': '31'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BusinessType'}, {'feature_label': 'BusinessType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BusinessType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Year', 'transformation_label': 'String Indexer'}], 'feature': 'Year', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '454', 'mean': '2019.56', 'stddev': '1.88', 'min': '2016', 'max': 'Web Services', 'missing': '46', 'distinct': '31'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Year'}, {'feature_label': 'Year', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Year')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'YearText', 'transformation_label': 'String Indexer'}], 'feature': 'YearText', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '498', 'mean': '2019.52', 'stddev': '1.87', 'min': '2016', 'max': 'Other', 'missing': '2', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'YearText'}, {'feature_label': 'YearText', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('YearText')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WashingtoniansAffectedRange', 'transformation_label': 'String Indexer'}], 'feature': 'WashingtoniansAffectedRange', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '844.35', 'stddev': '993.18', 'min': '1', 'max': 'Unknown', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'WashingtoniansAffectedRan...'}, {'feature_label': 'WashingtoniansAffectedRange', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'WashingtoniansAffectedRange')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BreachLifecycleRange', 'transformation_label': 'String Indexer'}], 'feature': 'BreachLifecycleRange', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '80.48', 'stddev': '386.2', 'min': '0', 'max': 'Unknown', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BreachLifecycleRange'}, {'feature_label': 'BreachLifecycleRange', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BreachLifecycleRange')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DaysToContainBreach', 'transformation_label': 'String Indexer'}], 'feature': 'DaysToContainBreach', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '476', 'mean': '696.4', 'stddev': '458.27', 'min': '0', 'max': 'Unknown', 'missing': '24', 'distinct': '32'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DaysToContainBreach'}, {'feature_label': 'DaysToContainBreach', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DaysToContainBreach')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DaysToIdentifyBreach', 'transformation_label': 'String Indexer'}], 'feature': 'DaysToIdentifyBreach', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '464', 'mean': '448.74', 'stddev': '451.16', 'min': '0', 'max': 'Unknown', 'missing': '36', 'distinct': '76'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DaysToIdentifyBreach'}, {'feature_label': 'DaysToIdentifyBreach', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DaysToIdentifyBreach')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DaysBreachLifecycle', 'transformation_label': 'String Indexer'}], 'feature': 'DaysBreachLifecycle', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '398', 'mean': '83.25', 'stddev': '161.68', 'min': '0', 'max': 'Unknown', 'missing': '102', 'distinct': '106'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DaysBreachLifecycle'}, {'feature_label': 'DaysBreachLifecycle', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DaysBreachLifecycle')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DiscoveredInProgress', 'transformation_label': 'String Indexer'}], 'feature': 'DiscoveredInProgress', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '406', 'mean': '133.88', 'stddev': '207.66', 'min': '0', 'max': 'True', 'missing': '94', 'distinct': '113'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DiscoveredInProgress'}, {'feature_label': 'DiscoveredInProgress', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DiscoveredInProgress')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DaysOfExposure', 'transformation_label': 'String Indexer'}], 'feature': 'DaysOfExposure', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '391', 'mean': '157.59', 'stddev': '217.72', 'min': '0', 'max': 'True', 'missing': '109', 'distinct': '156'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DaysOfExposure'}, {'feature_label': 'DaysOfExposure', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DaysOfExposure')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DaysElapsedBetweenEndAndDiscovery', 'transformation_label': 'String Indexer'}], 'feature': 'DaysElapsedBetweenEndAndDiscovery', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '396', 'mean': '146.13', 'stddev': '237.21', 'min': '0', 'max': 'True', 'missing': '104', 'distinct': '112'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DaysElapsedBetweenEndAndD...'}, {'feature_label': 'DaysElapsedBetweenEndAndDiscovery', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'DaysElapsedBetweenEndAndDiscovery')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'EndedOnDayDiscovered', 'transformation_label': 'String Indexer'}], 'feature': 'EndedOnDayDiscovered', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '401', 'mean': '123.11', 'stddev': '201.96', 'min': '0', 'max': 'True', 'missing': '99', 'distinct': '70'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'EndedOnDayDiscovered'}, {'feature_label': 'EndedOnDayDiscovered', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('EndedOnDayDiscovered')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DaysElapsedBeforeNotification', 'transformation_label': 'String Indexer'}], 'feature': 'DaysElapsedBeforeNotification', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '354', 'mean': '90.91', 'stddev': '184.0', 'min': '0', 'max': 'True', 'missing': '146', 'distinct': '127'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DaysElapsedBeforeNotifica...'}, {'feature_label': 'DaysElapsedBeforeNotification', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'DaysElapsedBeforeNotification')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DaysOfExposureRange', 'transformation_label': 'String Indexer'}], 'feature': 'DaysOfExposureRange', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '361', 'mean': '67.18', 'stddev': '65.3', 'min': '0', 'max': 'True', 'missing': '139', 'distinct': '70'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DaysOfExposureRange'}, {'feature_label': 'DaysOfExposureRange', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DaysOfExposureRange')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run washingtoniansHooks.ipynb
try:
	#sourcePreExecutionHook()

	databreachnotificationsaffecting = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Data_Breach_Notifications_Affecting.csv', 'filename': 'Data_Breach_Notifications_Affecting.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(databreachnotificationsaffecting)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run washingtoniansHooks.ipynb
try:
	#transformationPreExecutionHook()

	washingtoniansautofe = TransformationMain.run(databreachnotificationsaffecting,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "DateAware", "transformation_label": "String Indexer"}], "feature": "DateAware", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "479", "mean": "", "stddev": "", "min": "1/1/2020", "max": "9/9/2020", "missing": "21", "distinct": "319"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DateAware"}, {"transformationsData": [{"feature_label": "DateSubmitted", "transformation_label": "String Indexer"}], "feature": "DateSubmitted", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/10/2018", "max": "9/9/2020", "missing": "0", "distinct": "340"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DateSubmitted"}, {"transformationsData": [{"feature_label": "DataBreachCause", "transformation_label": "String Indexer"}], "feature": "DataBreachCause", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Cyberattack", "max": "Unauthorized Access", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataBreachCause"}, {"transformationsData": [{"feature_label": "DateStart", "transformation_label": "String Indexer"}], "feature": "DateStart", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "417", "mean": "", "stddev": "", "min": "1/1/2020", "max": "9/7/2019", "missing": "83", "distinct": "266"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DateStart"}, {"transformationsData": [{"feature_label": "DateEnd", "transformation_label": "String Indexer"}], "feature": "DateEnd", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "383", "mean": "", "stddev": "", "min": "1/1/2017", "max": "9/7/2020", "missing": "117", "distinct": "223"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DateEnd"}, {"transformationsData": [{"feature_label": "Name", "transformation_label": "String Indexer"}], "feature": "Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "20/20 Eye Care Network", "max": "thyssenkrupp Materials", "missing": "0", "distinct": "492"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Name"}, {"transformationsData": [{"feature_label": "Id", "transformation_label": "String Indexer"}], "feature": "Id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "10119.42", "stddev": "341.04", "min": " FSB", "max": "9775", "missing": "0", "distinct": "368"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Id"}, {"transformationsData": [{"feature_label": "CyberattackType", "transformation_label": "String Indexer"}], "feature": "CyberattackType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "424", "mean": "10123.67", "stddev": "359.61", "min": " Cherry River Farms", "max": "Unclear/unknown", "missing": "76", "distinct": "167"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CyberattackType"}, {"transformationsData": [{"feature_label": "WashingtoniansAffected", "transformation_label": "String Indexer"}], "feature": "WashingtoniansAffected", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "440", "mean": "28750.05", "stddev": "138559.37", "min": "1000", "max": "Unclear/unknown", "missing": "60", "distinct": "327"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "WashingtoniansAffected"}, {"transformationsData": [{"feature_label": "IndustryType", "transformation_label": "String Indexer"}], "feature": "IndustryType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "490", "mean": "29644.42", "stddev": "267067.77", "min": "1012", "max": "Ransomware", "missing": "10", "distinct": "152"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "IndustryType"}, {"transformationsData": [{"feature_label": "BusinessType", "transformation_label": "String Indexer"}], "feature": "BusinessType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "283", "mean": "1369.5", "stddev": "824.39", "min": "1690", "max": "Web Services", "missing": "217", "distinct": "31"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BusinessType"}, {"transformationsData": [{"feature_label": "Year", "transformation_label": "String Indexer"}], "feature": "Year", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "454", "mean": "2019.56", "stddev": "1.88", "min": "2016", "max": "Web Services", "missing": "46", "distinct": "31"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Year"}, {"transformationsData": [{"feature_label": "YearText", "transformation_label": "String Indexer"}], "feature": "YearText", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "498", "mean": "2019.52", "stddev": "1.87", "min": "2016", "max": "Other", "missing": "2", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "YearText"}, {"transformationsData": [{"feature_label": "WashingtoniansAffectedRange", "transformation_label": "String Indexer"}], "feature": "WashingtoniansAffectedRange", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "844.35", "stddev": "993.18", "min": "1", "max": "Unknown", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "WashingtoniansAffectedRan..."}, {"transformationsData": [{"feature_label": "BreachLifecycleRange", "transformation_label": "String Indexer"}], "feature": "BreachLifecycleRange", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "80.48", "stddev": "386.2", "min": "0", "max": "Unknown", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BreachLifecycleRange"}, {"transformationsData": [{"feature_label": "DaysToContainBreach", "transformation_label": "String Indexer"}], "feature": "DaysToContainBreach", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "476", "mean": "696.4", "stddev": "458.27", "min": "0", "max": "Unknown", "missing": "24", "distinct": "32"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DaysToContainBreach"}, {"transformationsData": [{"feature_label": "DaysToIdentifyBreach", "transformation_label": "String Indexer"}], "feature": "DaysToIdentifyBreach", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "464", "mean": "448.74", "stddev": "451.16", "min": "0", "max": "Unknown", "missing": "36", "distinct": "76"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DaysToIdentifyBreach"}, {"transformationsData": [{"feature_label": "DaysBreachLifecycle", "transformation_label": "String Indexer"}], "feature": "DaysBreachLifecycle", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "398", "mean": "83.25", "stddev": "161.68", "min": "0", "max": "Unknown", "missing": "102", "distinct": "106"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DaysBreachLifecycle"}, {"transformationsData": [{"feature_label": "DiscoveredInProgress", "transformation_label": "String Indexer"}], "feature": "DiscoveredInProgress", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "406", "mean": "133.88", "stddev": "207.66", "min": "0", "max": "True", "missing": "94", "distinct": "113"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DiscoveredInProgress"}, {"transformationsData": [{"feature_label": "DaysOfExposure", "transformation_label": "String Indexer"}], "feature": "DaysOfExposure", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "391", "mean": "157.59", "stddev": "217.72", "min": "0", "max": "True", "missing": "109", "distinct": "156"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DaysOfExposure"}, {"transformationsData": [{"feature_label": "DaysElapsedBetweenEndAndDiscovery", "transformation_label": "String Indexer"}], "feature": "DaysElapsedBetweenEndAndDiscovery", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "396", "mean": "146.13", "stddev": "237.21", "min": "0", "max": "True", "missing": "104", "distinct": "112"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DaysElapsedBetweenEndAndD..."}, {"transformationsData": [{"feature_label": "EndedOnDayDiscovered", "transformation_label": "String Indexer"}], "feature": "EndedOnDayDiscovered", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "401", "mean": "123.11", "stddev": "201.96", "min": "0", "max": "True", "missing": "99", "distinct": "70"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "EndedOnDayDiscovered"}, {"transformationsData": [{"feature_label": "DaysElapsedBeforeNotification", "transformation_label": "String Indexer"}], "feature": "DaysElapsedBeforeNotification", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "354", "mean": "90.91", "stddev": "184.0", "min": "0", "max": "True", "missing": "146", "distinct": "127"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DaysElapsedBeforeNotifica..."}, {"transformationsData": [{"feature_label": "DaysOfExposureRange", "transformation_label": "String Indexer"}], "feature": "DaysOfExposureRange", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "361", "mean": "67.18", "stddev": "65.3", "min": "0", "max": "True", "missing": "139", "distinct": "70"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DaysOfExposureRange"}]}))

	#transformationPostExecutionHook(washingtoniansautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run washingtoniansHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(washingtoniansautofe, ["DateAware_stringindexer", "DateSubmitted_stringindexer", "DataBreachCause_stringindexer", "DateStart_stringindexer", "DateEnd_stringindexer", "Name_stringindexer", "Id_stringindexer", "CyberattackType_stringindexer", "IndustryType_stringindexer", "BusinessType_stringindexer", "Year_stringindexer", "YearText_stringindexer", "WashingtoniansAffectedRange_stringindexer", "BreachLifecycleRange_stringindexer", "DaysToContainBreach_stringindexer", "DaysToIdentifyBreach_stringindexer", "DaysBreachLifecycle_stringindexer", "DiscoveredInProgress_stringindexer", "DaysOfExposure_stringindexer", "DaysElapsedBetweenEndAndDiscovery_stringindexer", "EndedOnDayDiscovered_stringindexer", "DaysElapsedBeforeNotification_stringindexer", "DaysOfExposureRange_stringindexer"], "WashingtoniansAffected_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

